In [1]:
#Get wikipedia contents
from nltk.stem.porter import *
import itertools
import os
import string
import wikipedia as wiki

def get_wikipedia_content_based_on_keyword_list(keywordlist, output_save_path):
    os.chdir(output_save_path)
    lst_keyword = keywordlist
    n_total = len(lst_keyword)
    file = open('wikioutputcombined.txt', 'w')
    for index, line in enumerate(lst_keyword):
        keyword = line.strip('\n').lower()
        print index, n_total, index * 1.0 / n_total, keyword
        try:
            content = wiki.page(keyword).content.encode('ascii', 'ignore')
        except wiki.exceptions.DisambiguationError as e:
            print 'DisambiguationError', keyword
        except:
            print 'Error', keyword
        if not content:
            continue
        for line in content.split('\n'):
            line = ' '.join(map(norm_word, line.split()))
            if line:
                file.write(line + '\n')
    file.close()


def get_wikipedia_content_based_on_keyword_list_one_file_per_keyword(keywordlist, output_save_path):
    os.chdir(output_save_path)
    lst_keyword = keywordlist
    n_total = len(lst_keyword)
    for index, line in enumerate(lst_keyword):
        keyword = line.strip('\n').lower()
        print index, n_total, index * 1.0 / n_total, keyword
        try:
            content = wiki.page(keyword).content.encode('ascii', 'ignore')
        except wiki.exceptions.DisambiguationError as e:
            print 'DisambiguationError', keyword
        except:
            print 'Error', keyword
        if not content:
            continue
        file = open('_'.join(keyword.split()) + '.txt', 'w')
        for line in content.split('\n'):
            line = ' '.join(map(norm_word, line.split()))
            if line:
                file.write(line + '\n')
        file.close()

def norm_word(word):
    word = word.lower().strip('?').strip('.').strip(',').strip('!').strip(':').strip(';').strip('\"').strip('\'')
    return word 

In [2]:
import nltk
from nltk.corpus import wordnet
import gensim
import logging
import os
import pandas as pd
import numpy as np
from scipy import spatial
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora
import re

class MyDocument(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            with open(os.path.join(self.dirname, fname)) as content_file:
                content = content_file.read()  
                content = unicode(content, encoding='utf-8', errors='replace')
                yield content.lower().split()

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = unicode(line, encoding='utf-8', errors='replace')
                yield line.lower().split()

def get_dictionary(path):
    dictionary = corpora.Dictionary( MySentences('/Users/MK/GitHub/the_answer_is/data/temporary2') )
    stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
    dictionary.filter_tokens(stop_ids) # remove stop words and words that appear only once
    dictionary.compactify() # remove gaps in id sequence after words that were removed
    return dictionary

def get_document_tuple( path, dictionary ):
    vector = pd.Series()
    doclist = MyDocument(path) 
    for i,content in enumerate(doclist):
        vector = vector.set_value(i,dictionary.doc2bow(content))
    return list(vector)

def transform_tuple_into_vector(document_tuple,dictionary):
    vector = pd.Series(np.zeros(len(dictionary.token2id)))        #0 array for storing wiki document vectors.
    if not document_tuple == []:
        for onetuple in document_tuple: 
            vector[onetuple[0]] = onetuple[1]
        vector = vector / np.linalg.norm(vector, ord = 1)                       #normalize vector     
        return vector 
    else:
        return vector

def transform_tuples_into_dataframe(document_tuples, dictionary):
    length = len(document_tuples) 
    vector_initialization = pd.Series(np.zeros(len(dictionary.token2id)))  #0 array for storing wiki document vectors. 
    df_vector = pd.DataFrame(vector_initialization)  #initialize dataframe. all vectors will be stored. 
    for i in xrange(len(document_tuples)):           
        #for each wiki documents, we will transform wiki vectors in tuple form into
        #vectors in ususal form. 
        vector = transform_tuple_into_vector( document_tuples[i], dictionary )
        df_vector[i]= pd.DataFrame(vector)
    return df_vector

def get_close_documents(string, dataframe, dictionary, topn):
    string = unicode(string, encoding='utf-8', errors='replace').lower()
    first_vector = transform_tuple_into_vector(dictionary.doc2bow(string.split()), dictionary )
    lengthlist = pd.Series()
    for j in xrange(len(dataframe.columns)):
        #lengthlist = lengthlist.set_value(j, np.linalg.norm(first_vector-dataframe[j]) )
        lengthlist = lengthlist.set_value( j, spatial.distance.cosine(first_vector, dataframe[j]))
    return lengthlist.sort_values().head(topn)

def get_document_by_index(path,index):
    # this gets the filename and content of the document in a directory by index. 
    i= 0
    for fname in os.listdir(path):
        if i == index:
            with open(os.path.join(path, fname)) as content_file:
                content = content_file.read()  
                content = unicode(content, encoding='utf-8', errors='replace')
                return fname, content
        i= i + 1
    return None

def print_names_of_close_documents(path, close_documents_list, outputpath):
    file = open(outputpath + '/close_documents.txt', 'w+')
    for i in close_documents_list.index:
        name, text = get_document_by_index(path,i)
        print name
        file.write(text + '\n' + '\n' + '\n' + '\n')        
    print 'text saved as txt'

def get_contents_of_close_documents_by_paragraph(path, close_documents_list):
    merged = ''
    for i in close_documents_list.index:
        name, text = get_document_by_index(path,i)
        merged = merged + '\n' + text        
    return merged.split('\n')


def get_distance_between_two_documents(A, B, dictionary):
    import re
    if A:
        A = re.sub(r'[^a-zA-Z ]',r'',A).lower().split()
    else:
        A = ['the']        # just in case A, or B is empty. 
    A = dictionary.doc2bow(A)
    A = transform_tuple_into_vector( A ,dictionary)
    
    if B:
        B = re.sub(r'[^a-zA-Z ]',r'',B).lower().split()
    else:
        B = ['the']       # just in case A, or B is empty. 
    B = dictionary.doc2bow(B)
    B = transform_tuple_into_vector( B ,dictionary)
    #length = spatial.distance.cosine(A,B)
    # I would like to use cosine distance, but the vectors are so sparse that most of the time the output is 0 .
    # So we use euclidean distance. 
    length = np.linalg.norm(A-B)
    return length


stoplist = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours',
             u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', 
             u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', 
             u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', 
             u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', 
             u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', 
             u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', 
             u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', 
             u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', 
             u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', 
             u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', 
             u'should', u'now']


#procedure needed for deleting words not in the training set. 
def only_the_words_in_index( list, index ):
    output = []
    for a, s in enumerate(list):
        if s in index:
            output.append(list[a])
    return output

def split_paragraph_list(paragraph_list):
    output = []
    for paragraph in paragraph_list:
        output.append( paragraph.split() )
    return output


def getdef(word):
    '''
    :param word: input is a word in a string format
    :return: returns a string that combines 3 definitions. 
    '''
    #addef = word.decode("utf8",'ignore')
    addef = word            #the word itself goes into the definition.

    for synset in wordnet.synsets(word)[:3]:
        try:
            addef = addef + ' ' + synset.definition()
        except:
            continue
    return addef

def doc_to_def(doc):
    ''' 
    :param doc: input is a document in a string format. Ex: "I have an apple" 
    :return: returns a list of definitions. The length of the list is the number of words in a string.   
    '''
    definition = []
    tokens = nltk.word_tokenize(doc)

    length = len(tokens)    # for printing out the progress
    #print 'doc_to_def progress:  0 / ', length
    progress = 0

    for element in tokens:
        temp = getdef(element)
        definition.append(temp)

        progress = progress + 1
        #print 'doc_to_def progress: ', progress, ' / ', length

    return definition

def split_definition(definition):
    split_def = []
    for index, line in enumerate(definition):
        line = line.lower()
        try:
            line = re.sub(r'[^a-zA-Z ]',r'',line)        #delete all but alphabet 
        except:
            print 'Error', line
        if not line:
            continue

        if line:                              # delete the empty lines. 
            line = line.split()
            #line = nltk.word_tokenize(line)
            line = [word for word in line if word not in stoplist]  # remove the stop words. 
            if line:
                split_def = split_def + line

    return split_def




In [9]:
dictionary_path = '/Users/MK/GitHub/the_answer_is/data/temporary2'  # question text from training data
dictionary = get_dictionary(dictionary_path)
keywordlist = list(dictionary.token2id)

In [10]:
#keywordlist = ['car','barack obama','France']
output_save_path = '/Users/MK/Desktop/Minchul/Machine Learning/Text Mining/get_wikipedia_contents_module_ck_data'
get_wikipedia_content_based_on_keyword_list_one_file_per_keyword(keywordlist, output_save_path)

0 5287 0.0 xylem
1 5287 0.000189143181388 limited
DisambiguationError limited
2 5287 0.000378286362777 similarity
DisambiguationError similarity
3 5287 0.000567429544165 phenotypical
4 5287 0.000756572725553 alleles
5 5287 0.000945715906942 magnetic
6 5287 0.00113485908833 saves
DisambiguationError saves
7 5287 0.00132400226972 desirable
DisambiguationError desirable
8 5287 0.00151314545111 crumpled
9 5287 0.00170228863249 yellow
10 5287 0.00189143181388 chaos
DisambiguationError chaos
11 5287 0.00208057499527 four
12 5287 0.00226971817666 tensile
13 5287 0.00245886135805 heliocentric
14 5287 0.00264800453944 controversial
15 5287 0.00283714772082 consists
DisambiguationError consists
16 5287 0.00302629090221 oldest
DisambiguationError oldest
17 5287 0.0032154340836 worked
DisambiguationError worked
18 5287 0.00340457726499 aggression
19 5287 0.00359372044638 poorly
20 5287 0.00378286362777 relationships
DisambiguationError relationships
21 5287 0.00397200680915 whose
22 5287 0.0041611

/Users/MK/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


TypeError: file() argument 1 must be encoded string without NULL bytes, not unicode

In [15]:
keywordlist = keywordlist[1:]
output_save_path = '/Users/MK/Desktop/Minchul/Machine Learning/Text Mining/get_wikipedia_contents_module_ck_data'
get_wikipedia_content_based_on_keyword_list_one_file_per_keyword(keywordlist, output_save_path)

0 3674 0.0 decades
1 3674 0.000272182906913 visiting
2 3674 0.000544365813827 texas
3 3674 0.00081654872074 happened
4 3674 0.00108873162765 chromatin
5 3674 0.00136091453457 carpal
6 3674 0.00163309744148 thereby
7 3674 0.00190528034839 matches
8 3674 0.00217746325531 conversations
9 3674 0.00244964616222 phytoplankton
10 3674 0.00272182906913 evolution
11 3674 0.00299401197605 sooner
12 3674 0.00326619488296 underneath
DisambiguationError underneath
13 3674 0.00353837778987 records
DisambiguationError records
14 3674 0.00381056069679 adrenaline
15 3674 0.0040827436037 markings
DisambiguationError markings
16 3674 0.00435492651062 base
DisambiguationError base
17 3674 0.00462710941753 half
18 3674 0.00489929232444 feature
DisambiguationError feature
19 3674 0.00517147523136 bath
DisambiguationError bath
20 3674 0.00544365813827 killed
DisambiguationError killed
21 3674 0.00571584104518 maintaining
22 3674 0.0059880239521 highlevel
DisambiguationError highlevel
23 3674 0.00626020685901

TypeError: file() argument 1 must be encoded string without NULL bytes, not unicode